In [1]:
from validate_coarse_to_fine import * 
from dataloader.BSD500 import BSD500
import torch
from R_network_mm import RNet
import json
from matplotlib import pyplot as plt
from torchmetrics.functional import peak_signal_noise_ratio as psnr
from torchmetrics.functional import structural_similarity_index_measure as ssim

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 5

val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_5_mm_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]

exp = "/home/pourya/mm_final/exps/sigma5/64_2_7_1e-3_mm"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:1'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()

model.W2.W1.weight.data = model.sumtoone(model.W2.W1.weight)
for i, _ in enumerate(model.W2.W1s): 
    model.W2.W1s[i].weight.data = model.sumtoone(model.W2.W1s[i].weight)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)


model.eigenimage = model.eigenimage.to(device)
print(model.lmbda, p1)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


Parameter containing:
tensor([[0.0006]], device='cuda:1', requires_grad=True) 0.000534174614761


In [ ]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)

    return c_k

In [ ]:
i = 0
psnrs = list()

with torch.no_grad():
    for img in test_dataset:
        i = i + 1
        if True:
            gt = img.to(device)[None, :, :, :]
            noisy_image = (img.to(device) + (5/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
            denoised =safi_denoise(noisy_image)
            psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
            psnrs.append(psnr)
            print(i, psnr)

1 36.41
2 37.46
3 38.15
4 36.93
5 35.3
6 37.95
7 37.82
8 36.86
9 40.74
10 40.29
11 35.67
12 35.41
13 40.4
14 36.11
15 38.13
16 36.05
17 38.25
18 37.62
19 40.12
20 37.11
21 38.15
22 36.95
23 37.82
24 37.56
25 38.24
26 36.97
27 36.54
28 39.04
29 38.83
30 38.09
31 34.85
32 35.14
33 36.09
34 41.02
35 36.31
36 37.72
37 40.62
38 36.99
39 38.91
40 38.24
41 37.78
42 37.05
43 38.86
44 35.94
45 34.97
46 36.13
47 36.94
48 37.29
49 36.44
50 40.88
51 44.74
52 36.82
53 38.68
54 35.49
55 38.99
56 38.37
57 37.44
58 36.08
59 37.41
60 36.94
61 39.25
62 37.84
63 38.81
64 37.52
65 34.77
66 37.58
67 37.57
68 38.01


In [ ]:
print(np.round(np.mean(np.array(psnrs)), 2), np.round(np.std(np.array(psnrs)), 2))

37.67 1.72
